# **Statistical foundation of machine learning**

General description of the problem here

# Dataset preparation & cross-validation strategy

We load and organize the raw data as follows:

1. *Guided Dataset*:
    - a numpy array `X_guided` of shape `(5, 8, 230000)`;
    - a numpy array `Y_guided` of shape `(5, 51, 230000)`,

2. *Freemoves Dataset*:
    - a numpy array `X_freemoves` of shape `(5, 8, 270000)`;
    - a numpy array `Y_freemoves` of shape `(5, 8, 270000)`.
    
The arrays `X_guided` and `X_freemoves` each consist of five sessions of eight raw EMG signals, with lengths of `230000` and `270000` samples, respectively. Their counterparts, `Y_guided` and `Y_freemoves`, hold the synchronized time‑series for 51 joint‑angle components (17 joints × 3 components) organized over the same five sessions. The data is sampled at 1024Hz, so 1024 samples correspond to one second of data.

In [ ]:
# loading raw data
import numpy as np

PATH = f'/Users/marco/PROJECTS/data/'
# PATH = r'C:\Users\gianm\Documents\Uni\Big Data\F422\project\data\\'

DATASET = 'guided'
X_guided = np.load(PATH + f'{DATASET}/{DATASET}_dataset_X.npy')         # shape (5, 8, 230000)
Y_guided = np.load(PATH + f'{DATASET}/{DATASET}_dataset_Y.npy')         # shape (5, 51, 230000)

DATASET = 'freemoves'
X_freemoves = np.load(PATH + f'{DATASET}/{DATASET}_dataset_X.npy')      # shape (5, 8, 270000)
Y_freemoves = np.load(PATH + f'{DATASET}/{DATASET}_dataset_Y.npy')      # shape (5, 51, 270000)

The main objective of this project is to predict the joint angle data at time `t` in terms of a portion of the EMG signals ending at time `t`.

To prepare our dataset for this regression tasks, we segment the EMG signals into overlapping windows of fixed size. This preprocessing step is crucial as it enables the learning models to capture localized temporal patterns in muscle activity and link them to corresponding hand pose estimations.

We choose a **window size of 500 samples**, which corresponds to roughly **0.49 seconds** of EMG data, given the sampling rate of **1024 Hz**. This window size strikes a balance between capturing enough signal dynamics and ensuring real-time usability for prosthesis control.

We implement a **50% overlap**, meaning each window starts 250 samples after the previous one. This overlap increases the number of training samples without excessively inflating computational costs. Larger overlaps (e.g., 75%) generate even more samples but demand significantly more memory and processing time, which may not scale efficiently depending on available resources.

This approach is implemented using the custom transformer classes `TimeWindowTransformer` and `LabelWindowExtractor`, implemented and documented in the module `config.transformers`. To initialize instances of these classes, we need to pass two parameters:

- `size`: the size of the time windows (in our case, typically `size=500`);

- `step`: the step separating two consecutive time windows (in our case, at train/evaluation time we will use `step = 250`, but at prediction time we will use a smaller value to obtain more precise predictions).

The most important method of these transformers is the `.transform()` method. For example:

- For an instance `tw_transformer` of `TimeWindowTransformer`, the `.transform()` method takes in input a numpy array `X` of raw EMG signals (shape `(..., n_channels, n_times)`) and returns a numpy array `X_windows` of time windows (shape `(...,n_windows, n_channels, window_size)`);

- For an instance `label_extractor` of `LabelWindowExtractor`, the `.transform()` method takes in input a numpy array `Y` of time-series of joint angle vectors (shape `(..., n_angles, n_times)`) and returns a numpy array `Y_labels` extracting the last joint angle vector for each time window (shape `(...,n_windows, n_angles)`).

These transformers are easier to use than to describe! Below we have an explicit example.

In [ ]:
from config.transformers import TimeWindowTransformer, LabelWindowExtractor

# define parameters
size = 500
step = 250

# initialize transformers
tw_transformer = TimeWindowTransformer(size=size, step=step)
label_extractor = LabelWindowExtractor(size=size, step=step)

# apply transformations
X_guided_windows = tw_transformer.transform(X_guided)           # shape: (5, n_windows, 8, 500)
Y_guided_labels = label_extractor.transform(Y_guided)           # shape: (5, n_windows, 51)

X_freemoves_windows = tw_transformer.transform(X_freemoves)     # shape: (5, n_windows, 8, 500)
Y_freemoves_labels = label_extractor.transform(Y_freemoves)     # shape: (5, n_windows, 51)

### Cross-validation strategy

The regression problem can be formulated schematically as follows: given a time window `X` of EMG data (shape `(8, 500)`), we need to predict the vector of joint angles (shape `(51,)` ) corresponding to the hand configuration at the end of the time window.

We can therefore consider each of the *guided* and *freemoves* datasets as a dataset of `N` labeled pairs `(X,y)`, where `X` is a EMG time window and `y` is its corresponding vector of joint angles. The data is organized across *five sessions*.

It is **not appropriate** to naively stack these sessions together and then perform cross-validation on the stacked dataset, as this can lead to **leakage**. The problem is that some time windows overlap; with a naive cross-validation on the stacked dataset, we risk having pairs `(X,y)` in the validation set which overlap with pairs in the training set.

To rigorously evaluate the performance and generalization capabilities of our regression models, we instead implement a **Leave-One-Session-Out cross-validation strategy** that ensures independence between training and validation data.

The available data consists of 5 sessions. We use the first **4 sessions** (referred to as the `train_val` part of the data) for training and validation, and **reserve the 5th session** as completely unseen data (referred to as the `test` part of the data). This separation allows us to evaluate and tune our models on the `train_val` part of the data, and then finally evaluate the ability of our models to generalize by measuring their performance on the `test` session.

We will train and validate our models on the `train_val` portion of the data by adopting a **Leave-One-Session-Out 4-fold cross-validation** strategy. Specifically, we will repeatedly train our model on three out of the four `train_val` sessions, and then validate the model on the remaining fourth `train_val` session, for each possible choice of the validation session. The performance is then evaluated in terms of two metrics:

- the average of the *Root Mean Squared Errors* (**RMSE**) across folds;
- the average of the *Normalized Mean Squared Errors* (**NMSE**) across folds.

We will cross-validate our `sklearn`-compatible models by means of the custom `cross_validate_pipeline` function, implemented and documented in the module `config.validation`.

In [ ]:
# separate between train_val and test datasets
X_guided_windows_train_val_folds = X_guided_windows[:4]         # shape (4, N, 8, 500)
Y_guided_labels_train_val_folds = Y_guided_labels[:4]           # shape (4, N, 51)
X_guided_windows_test = X_guided_windows[4]                     # shape (N, 8, 500)
Y_guided_labels_test = Y_guided_labels[4]                       # shape (N, 51)

X_freemoves_windows_train_val_folds = X_freemoves_windows[:4]   # shape (4, N, 8, 500)
Y_freemoves_labels_train_val_folds = Y_freemoves_labels[:4]     # shape (4, N, 51)
X_freemoves_windows_test = X_freemoves_windows[4]               # shape (N, 8, 500)
Y_freemoves_labels_test = Y_freemoves_labels[4]                 # shape (N, 51)

# Baseline pipelines

Create a custom class inheriting from scikit-learn’s `BaseEstimator`
and `TransformerMixin` that implements the extraction of common time-domain features described
in section 3.1. Note that the features described in Section 3.1 represent the minimal required set. We
encourage you to include additional features or preprocessing steps if you would like to further improve your model performances. Select at least two different regression models, compare their cross-validated performance, and evaluate their feature importances. For both models, perform feature selection to determine the optimal subset of features minimizing the Root Mean Squared Error (RMSE).
Clearly document this process in your notebook, discussing the outcomes in detail. Finally, create a
scikit-learn `Pipeline` that integrates your custom feature extraction class, the optimal feature selection step, and the best-performing regression model identified from your cross-validation results.
Using visualizations and tables to illustrate your findings, and employing formulas or pseudo-code
to explain the feature selection procedure, is strongly encouraged. Note that one-third of the score
will depend on the quality and clarity of your documentation.

### Time-domain features

To establish a robust performance benchmark, we design a baseline approach that uses standard time-domain features extracted from the raw EMG signal windows. These features have been widely used in EMG-based classification and regression problems, due to their low computational cost and effectiveness in capturing muscle activity dynamics.

In order to extract time-domain features, we implement a custom class `TimeDomainTransformer`, inheriting from `BaseEstimator` and `TransformerMixin`. This class is implemented and documented in `config.transformers`.

An instance of `TimeDomainTransformer` has two main methods:

- a `.fit(X, y)` method: this method does not really do anything, it simply returns the instance of the custom class; it is necessary for compatibility with sklearn's pipeline usage;

- a `.transform(X)` method: this method takes in input a numpy array of EMG signals of shape `(...,n_channels, n_times)` (in our case, typically `n_channels = 8` and `n_times = 500`), and returns a numpy array of shape `(...,12)` containing 12 key time-domain features from each EMG signal, for a total of **96 features**.

We implement the following 12 aggregate statistics:

- **MAV** – Mean Absolute Value
- **RMS** – Root Mean Square
- **VAR** – Variance
- **STD** – Standard Deviation
- **ZC** – Zero Crossing Count
- **MPR** – Myopulse Percentage Rate (using a tunable threshold `sigma_mpr`)
- **MAA** – Maximum Absolute Amplitude
- **WL** – Waveform Length
- **SSC** – Slope Sign Changes
- **WA** – Wilson Amplitude
- **MFL** – Maximum Fractal Length
- **KRT** – Kurtosis

Below is a usage example.

In [ ]:
from config.transformers import TimeDomainTransformer

tw_transformer = TimeDomainTransformer() # initialize instance of the time-domain transformer

array_of_EMG_signals = X_guided_windows_train_val_folds[0]                  # N time windows, shape (N, 8, 500)
array_of_time_features = tw_transformer.transform(array_of_EMG_signals)     # N feature vectors, shape (N, 96)

In [ ]:
# visualization via pandas
import pandas as pd
import itertools

feature_names = ['MAV', 'RMS', 'VAR', 'STD', 'ZC', 'MPR', 'MAA', 'WL', 'SSC', 'WA', 'MFL', 'KRT']

df = pd.DataFrame(
    data = array_of_time_features,
    columns = [feature + f'_{channel}' for channel, feature in itertools.product(np.arange(8), feature_names)]
    )

df.head(10)

### Baseline estimators

We will now construct three sklearn-compatible pipelines, using different regression algorithms to predict joint angles from time-domain features extracted from EMG time windows. We will then evaluate the cross-validated performance of those pipelines. These will be our *baseline* results, on which we will compare more complex models. Later, we will improve the baseline results by:

- adding more layers to the pipelines (eg. *filters* and *feature selection transformers*);
- developing new pipelines based on different features extracted from EMG signals, such as *covariance matrices*;
- developing *convolutional neural networks*, working directly on EMG time windows;
- wrap the previously defined model together into *ensemble methods*.

Each baseline pipeline follows the same general structure:

1. **Feature Extraction**  
   Extracts 12 time-domain features from each EMG channel, using the transformer `TimeDomainTransformer` described above.

2. **Standardization**  
   Normalizes the extracted features using the `sklearn` transformer `StandardScaler`, to ensure that each feature contributes equally to the regression model.

3. **Regression Model**  
   Each pipeline uses a different learning algorithm:
   - **Kernel Ridge Regression**
   - **K-Nearest Neighbors Regression**
   - **Random Forest Regression**

We chose these three regression models for two main reasons:
* **Empirical reason:** We tested various regression models available in the sklearn library, and these three seemed to have a good performance compared to the others; for reasons of space, we did not include our numerous attempts in this notebook.

* **Expected non-correlation:** Later in the notebook, we will combine our baseline estimators into *ensemble models*. It is well-known that ensemble models work well when the base learners are not very correlated to each other. The three regressors we chose are based on different regression methods:
    - **Ridge Regression** is a *linear model*, adapted to our non-linear problem by means of the *kernel trick*;

    - **KNN** is a *distance-based* model;

    - **Random Forest** is a *tree-based* ensemble model.
  
  By choosing base learners which adopt different regression strategies, we hope to achieve significant improvement by employing ensemble strategies later.

> **DISCLAIMER**

All models and pipelines developed in this project adhere to the following input/output logic, to ensure compatibility with the dataset and evaluation routines:

- `model.fit(X, y)`  
  - **Input:**  
    - `X`: numpy array of shape `(N, n_channels, n_times)` — N windows of raw EMG signals of length `n_times`, organized in `n_channels` channels (in our case, typically `n_channels = 8` and `n_times = 500`);
    - `y`: numpy array of shape `(N, n_angles)` — corresponding N vectors of `n_angles` joint-angle labels (in our case, typically `n_angles = 51`).
  - **Note:** This method fits the model in place; the return value is not used.

- `model.predict(X)`  
  - **Input:** `X`: numpy array of shape `(N, n_channels, n_times)`;
  - **Output:** numpy array of shape `(N, n_angles)`.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.kernel_ridge import KernelRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
baseline_kr = Pipeline(
    [
        ('feature_extraction', TimeDomainTransformer(sigma_mpr=0.3)),
        ('scaler', StandardScaler()),
        ('regressor', KernelRidge(
            alpha = 0.01,
            gamma = 0.01,
            kernel='laplacian'))
    ]
)

baseline_knn = Pipeline(
    [
        ('feature_extraction', TimeDomainTransformer(sigma_mpr=0.3)),
        ('scaler', StandardScaler()),
        ('regressor', KNeighborsRegressor(
            n_neighbors = 5))
    ]
)

baseline_rf = Pipeline(
    [
        ('feature_extraction', TimeDomainTransformer(sigma_mpr=0.3)),
        ('scaler', StandardScaler()),
        ('regressor', RandomForestRegressor(
            n_estimators = 50,
            max_depth = 10))
    ]
)

### Baseline evaluation via cross-validation

To evaluate our baseline models, we apply the **4-fold cross-validation** strategy described previously. For each model, we report both:

- average **Root Mean Squared Error (RMSE)** across folds;

- average **Normalized Mean Squared Error (NMSE)** across folds.

To compute these metrics, we use the functions `RMSE` and `NMSE` implemented and documented in the module `config.validation`.

Before diving into the full cross-validation report, let's first provide a manual fit/predict example run.

In [ ]:
from config.validation import RMSE, NMSE

# prepare training and validation data
X_train = np.vstack(X_guided_windows_train_val_folds[:3])   # stacked first 3 sessions, shape (3*N, 8, 500)
Y_train = np.vstack(Y_guided_labels_train_val_folds[:3])    # stacked first 3 sessions, shape (3*N, 51)

X_val = X_guided_windows_train_val_folds[3]                 # validation session, shape (N, 8, 500)
Y_val = Y_guided_labels_train_val_folds[3]                 # validation session, shape (N, 51)

# choose an example model
model = baseline_knn

# fit the model
model.fit(X_train, Y_train) # fits the model in-place

# compute predictions
Y_pred = model.predict(X_val) # shape (N, 51)

# compute errors
rmse_error = RMSE(Y_pred, Y_val)
nmse_error = NMSE(Y_pred, Y_val)

print(f'RMSE error: {rmse_error:.2f}')
print(f'NMSE error: {nmse_error:.2f}')

We can now proceed with full-fledged cross-validation.

#### 4-fold Leave-One-Session-Out cross-validation of baseline models on *guided* dataset

In [ ]:
from config.validation import cross_validate_pipeline, RMSE, NMSE # custom cross-validation function

In [ ]:
print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "baseline_kr" on the guided dataset:')
results_guided_baseline_kr = cross_validate_pipeline(
    pipeline = baseline_kr,
    X_folds = X_guided_windows_train_val_folds,
    Y_folds = Y_guided_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "baseline_knn" on the guided dataset:')
results_guided_baseline_knn = cross_validate_pipeline(
    pipeline = baseline_knn,
    X_folds = X_guided_windows_train_val_folds,
    Y_folds = Y_guided_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "baseline_rf" on the guided dataset:')
results_guided_baseline_rf = cross_validate_pipeline(
    pipeline = baseline_rf,
    X_folds = X_guided_windows_train_val_folds,
    Y_folds = Y_guided_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

#### 4-fold Leave-One-Session-Out cross-validation of baseline models on *freemoves* dataset

In [ ]:
print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "baseline_kr" on the freemoves dataset:')
results_freemoves_baseline_kr = cross_validate_pipeline(
    pipeline = baseline_kr,
    X_folds = X_freemoves_windows_train_val_folds,
    Y_folds = Y_freemoves_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "baseline_knn" on the freemoves dataset:')
results_freemoves_baseline_knn = cross_validate_pipeline(
    pipeline = baseline_knn,
    X_folds = X_freemoves_windows_train_val_folds,
    Y_folds = Y_freemoves_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "baseline_rf" on the freemoves dataset:')
results_freemoves_baseline_rf = cross_validate_pipeline(
    pipeline = baseline_rf,
    X_folds = X_freemoves_windows_train_val_folds,
    Y_folds = Y_freemoves_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

# Features selection on baseline pipeline

In order to perform feature selection to determine the optimal subset of features minimizing the Root Mean Squared Error (RMSE) for hand pose estimation, we implemented two approaches: **mRMR (Minimum Redundancy Maximum Relevance)** and **PCA**. Feature selection might help us to mitigate the curse of dimensionality, reducing the risk of overfitting our regression models, and identify the most informative features from the extracted time-domain characteristics of the EMG signals.

## mRMR
We want to start exploring the feature selection with the use of **Minimum Redundancy Maximum Relevance (mRMR)** strategy, based on a correlation-based approximation of mutual information. The core idea of mRMR is to select features that have high individual correlation with the target variables (maximum relevance) while having low correlation with each other (minimum redundancy). This helps in building a more robust and interpretable model.
To estimate the relevance of each feature to the outputs (joint angles), we approximate **Mutual Information (MI)** using the correlation coefficient:

$$
\text{MI}(x, y) \approx -\frac{1}{2} \log(1 - \rho^2)
$$

Where:
- $\rho$ is the **Pearson correlation coefficient**
- This approximation assumes **joint Gaussianity**

Here's a sketch of the implementation:
```python
def mutual_info_corr(x, y):
    """
    Approximates mutual information between x and y via Pearson correlation.
    """
    c = np.corrcoef(x, y)[0, 1]
    if abs(c) == 1:
        c = 0.999999  # avoid division by zero
    return -0.5 * np.log(1 - c**2)
```

The mRMR algorithm iteratively selects the next best feature that maximizes the difference between its relevance to the outputs and its redundancy with the already selected features:

$$
\text{mRMR}_j = \text{Relevance}_j - \text{Redundancy}_j
$$

Where, for a candidate feature *j*:
- **Relevance**: average MI between the candidate feature *j* and all the output variables
- **Redundancy**: average MI between the candidate feature *j* and all the features that have already been selected in previous iterations

Here's a sketch of the implementation:
```python
def greedy_mrmr_selection(X_df, mi_scores):
    selected = []
    candidates = list(range(X_df.shape[1]))

    for _ in range(len(candidates)):
        redundancy = ...
        mrmr_score = mi_scores[candidates] - redundancy
        best = candidates[np.argmax(mrmr_score)]
        selected.append(best)
        candidates.remove(best)
    return selected
```

To evaluate the performance of the feature subsets selected by mRMR, we performed the following steps:

1.  **Iterate through Feature Subset Sizes:** For each subset of selected features, starting from a size of 1 up to $N$ (the total number of extracted features):
    * **Create Feature Subset:** Construct a new feature matrix containing only the top $k$ selected features, where $k$ is the current subset size.
    * **Build Pipeline:** Create a scikit-learn `Pipeline` that includes a chosen regressor from the described baseline (e.g., `Ridge`, `RandomForestRegressor`).
    * **Cross-Validation:** Employ `cross_validate_pipeline` with a $4$-fold strategy to compute the average RMSE across the leave-one-session-out cross-validation folds for this pipeline configuration
    * **Calculate Performance Metrics:** Compute the mean and standard deviation of the RMSE values obtained from the cross-validation folds.

2.  **Visualize Results:** After evaluating the performance for all feature subset sizes, we generate a plot to visualize the relationship between the number of features and the prediction accuracy. Vertical error bars are plotted around each mean RMSE point to indicate the standard deviation of the RMSE across the different cross-validation folds. This provides a measure of the variability in performance. Each data point in the plot is colored based on the type of the time-domain feature that was added at that specific step in the mRMR selection process (e.g., all 'Mean Absolute Value' (MAV) features have one color, all 'Root Mean Square' (RMS) features another, and so on). 

This systematic evaluation of mRMR-selected feature subsets aims to identify an effective set of EMG time-domain features that can accurately predict hand joint angles while minimizing redundancy.

In [ ]:
from config.validation import mutual_info_corr, compute_mi_vector, greedy_mrmr_selection
from config.transformers import FeatureSelector
from config.utils import plot_mrmr_results
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Evaluation dataset guided

# Compute Mutual Info (includes feature extraction)
X_df, Y_all, mi_scores, X_sessions = compute_mi_vector(
    X_guided_windows_train_val_folds, 
    Y_guided_labels_train_val_folds, 
    TimeDomainTransformer(sigma_mpr=0.3)
    )

# Perform mRMR
selected = greedy_mrmr_selection(X_df, mi_scores)

models_feat_sel = {
    'Kernel Ridge': KernelRidge(
        alpha=0.01, 
        gamma=0.01, 
        kernel='laplacian'),
    'KNN': KNeighborsRegressor(n_neighbors=5),
    'Random Forests': RandomForestRegressor(
        n_estimators=50, 
        max_depth=10)
}
n_features = len(selected)
n_sessions = X_sessions.shape[0]
CV_results = {}

for model_name, model in models_feat_sel.items():
    print('---------------------------------------------------------------------------')
    print(f'Evaluating mMRM feature selection of the model {model_name} on the guided dataset...')
    CV_err_mrmr = np.zeros((n_features, n_sessions))
    
    for nb_features in range(1, n_features + 1):
        selected_indices = selected[:nb_features]
        pipeline = Pipeline([
            ('select', FeatureSelector(selected_indices)),
            ('scale', StandardScaler()),
            ('reg', model)
        ])
        try:
            cv_results = cross_validate_pipeline(
                pipeline,
                X_sessions,
                Y_guided_labels_train_val_folds,
                metric_fns={'RMSE': RMSE},
                n_folds=n_sessions,
                verbose=0
            )
        except Exception as e:
            print(f"Failed for model: {model_name} | #features: {nb_features}")
            print(e)
            break
        for fold_id in range(n_sessions):
            CV_err_mrmr[nb_features - 1, fold_id] = cv_results[fold_id]['val_RMSE']
    
    CV_results[model_name] = CV_err_mrmr
    
    # Plot results for the current model
    plot_mrmr_results(
        selected,
        CV_err_mrmr,
        td_feature_names=['MAV', 'RMS', 'VAR', 'STD', 'ZC', 'MPR',
                          'MAA', 'WL', 'SSC', 'WA', 'MFL', 'KRT'],
        title=f'mRMR Feature Selection - {model_name} - guided dataset'
    )

# If needed for inspection
# Report results (optional - you might not need this if you have the plots)
# for model_name, CV_err_mrmr in CV_results.items():
#     print(f"\nMRMR Feature Selection with {model_name}")
#     for i in range(n_features):
#         print(f"#Features: {i + 1}; CV error = {CV_err_mrmr[i, :].mean():.4f}; std dev = {CV_err_mrmr[i, :].std():.4f}")

In [ ]:
# Evaluation dataset freemoves

# Compute Mutual Info (includes feature extraction)
X_df, Y_all, mi_scores, X_sessions = compute_mi_vector(
    X_freemoves_windows_train_val_folds, 
    Y_freemoves_labels_train_val_folds, 
    TimeDomainTransformer(sigma_mpr=0.3)
    )

# Perform mRMR
selected = greedy_mrmr_selection(X_df, mi_scores)

# Models
models_feat_sel = {
    'Kernel Ridge': KernelRidge(
        alpha=0.01, 
        gamma=0.01, 
        kernel='laplacian'),
    'KNN': KNeighborsRegressor(n_neighbors=5),
    'Random Forests': RandomForestRegressor(
        n_estimators=50, 
        max_depth=10)
}
n_features = len(selected)
n_sessions = X_sessions.shape[0]
CV_results = {}

for model_name, model in models_feat_sel.items():
    print(f"\nEvaluating model: {model_name}")
    print('---------------------------------------------------------------------------')
    print(f'Evaluating mMRM feature selection of the model {model_name} on the freemoves dataset...')
    CV_err_mrmr = np.zeros((n_features, n_sessions))
    
    for nb_features in range(1, n_features + 1):
        selected_indices = selected[:nb_features]
        pipeline = Pipeline([
            ('select', FeatureSelector(selected_indices)),
            ('scale', StandardScaler()),
            ('reg', model)
        ])
        try:
            cv_results = cross_validate_pipeline(
                pipeline,
                X_sessions,
                Y_freemoves_labels_train_val_folds,
                metric_fns={'RMSE': RMSE},
                n_folds=n_sessions,
                verbose=0
            )
        except Exception as e:
            print(f"Failed for model: {model_name} | #features: {nb_features}")
            print(e)
            break
        for fold_id in range(n_sessions):
            CV_err_mrmr[nb_features - 1, fold_id] = cv_results[fold_id]['val_RMSE']
    
    CV_results[model_name] = CV_err_mrmr
    
    # Plot results for the current model
    plot_mrmr_results(
        selected,
        CV_err_mrmr,
        td_feature_names=['MAV', 'RMS', 'VAR', 'STD', 'ZC', 'MPR',
                          'MAA', 'WL', 'SSC', 'WA', 'MFL', 'KRT'],
        title=f'mRMR Feature Selection - {model_name} - freemoves dataset'
    )

# If needed for inspection
# Report results (optional - you might not need this if you have the plots)
# for model_name, CV_err_mrmr in CV_results.items():
#     print(f"\nMRMR Feature Selection with {model_name}")
#     for i in range(n_features):
#         print(f"#Features: {i + 1}; CV error = {CV_err_mrmr[i, :].mean():.4f}; std dev = {CV_err_mrmr[i, :].std():.4f}")

We evaluated RMSE across varying numbers of selected features using leave-one-session-out cross-validation. Here are the results near the optimal region (number reported for Kernel Ridge):

After visualizing the performance of different feature subset sizes using the plots titled "mRMR Feature Selection - [Model] - guided dataset", we examined the results to identify the optimal number of features. As an example, for the Kernel Ridge model, the RMSE across different numbers of selected features using leave-one-session-out cross-validation showed the following trend near the optimal region:

```text
#Features: 80; CV error = 5.2161; std dev = 0.6287  
#Features: 81; CV error = 5.2204; std dev = 0.6276  
#Features: 82; CV error = 5.2140; std dev = 0.6210  
#Features: 85; CV error = 5.2095; std dev = 0.6255  
#Features: 90; CV error = 5.1963; std dev = 0.6175  
#Features: 95; CV error = 5.1783; std dev = 0.6114  
#Features: 96; CV error = 5.1763; std dev = 0.6115
```

The results show that after 80 features, additional features provide the addition of more features yields only minimal improvements in the cross-validation RMSE,

To integrate feature selection directly into the learning pipeline, we implemented the `TopKMRMRSelector` class as a custom scikit-learn compatible transformer. This enables model training and evaluation without requiring separate preprocessing steps.
By setting k=80, we aim to utilize only the top 80 most relevant features as identified by the mRMR algorithm, improving interpretability and reducing the dimensionality the dimensionality of the feature space from the original 96 dimensions.

In [ ]:
from config.transformers import TopKMRMRSelector

baseline_mrmr_kr = Pipeline(
    [
        ('feature_extraction', TimeDomainTransformer()),
        ('scaler', StandardScaler()),
        ('feature_select', TopKMRMRSelector(k=80)),
        ('regressor', KernelRidge(
            alpha = 0.01,
            gamma = 0.01,
            kernel='laplacian'))
    ]
)

baseline_mrmr_knn = Pipeline(
    [
        ('feature_extraction', TimeDomainTransformer()),
        ('scaler', StandardScaler()),
        ('feature_select', TopKMRMRSelector(k=80)),
        ('regressor', KNeighborsRegressor(
            n_neighbors = 5))
    ]
)

baseline_mrmr_rf = Pipeline(
    [
        ('feature_extraction', TimeDomainTransformer()),
        ('scaler', StandardScaler()),
        ('feature_select', TopKMRMRSelector(k=80)),
        ('regressor', RandomForestRegressor(
            n_estimators = 50,
            max_depth = 10))
    ]
)

In [ ]:
print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "baseline_mrmr_kr" on the guided dataset:')
results_guided_baseline_mrmr_kr = cross_validate_pipeline(
    pipeline = baseline_mrmr_kr,
    X_folds = X_guided_windows_train_val_folds,
    Y_folds = Y_guided_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "baseline_mrmr_knn" on the guided dataset:')
results_guided_baseline_mrmr_knn = cross_validate_pipeline(
    pipeline = baseline_mrmr_knn,
    X_folds = X_guided_windows_train_val_folds,
    Y_folds = Y_guided_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "baseline_mrmr_rf" on the guided dataset:')
results_guided_baseline_mrmr_rf = cross_validate_pipeline(
    pipeline = baseline_mrmr_rf,
    X_folds = X_guided_windows_train_val_folds,
    Y_folds = Y_guided_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

#### 4-fold Leave-One-Session-Out cross-validation of baseline models + mRMR on *freemoves* dataset

In [ ]:
print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "baseline_mrmr_kr" on the freemoves dataset:')
results_freemoves_baseline_mrmr_kr = cross_validate_pipeline(
    pipeline = baseline_mrmr_kr,
    X_folds = X_freemoves_windows_train_val_folds,
    Y_folds = Y_freemoves_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "baseline_mrmr_knn" on the freemoves dataset:')
results_freemoves_baseline_mrmr_knn = cross_validate_pipeline(
    pipeline = baseline_mrmr_knn,
    X_folds = X_freemoves_windows_train_val_folds,
    Y_folds = Y_freemoves_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "baseline_mrmr_rf" on the freemoves dataset:')
results_freemoves_baseline_mrmr_rf = cross_validate_pipeline(
    pipeline = baseline_mrmr_rf,
    X_folds = X_freemoves_windows_train_val_folds,
    Y_folds = Y_freemoves_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

## PCA

We will now turn our attention to the **Principal Component Analysis (PCA)** and how it can be used as a feature selection and dimensionality reduction technique to improve regression performance from EMG time-domain features.

We evaluate PCA-based dimensionality reduction by retaining different levels of explained variance, defined by the following thresholds:

```python
variance_thresholds = np.arange(0.85, 0.991, 0.01)
```

For each baseline regressor we:
- Build a scikit-learn pipeline with the regressor with a PCA step where the number of components is determined by the current variance_threshold
- Use cross_validate_pipeline() with a $4$-fold strategy to compute the average RMSE across the leave-one-session-out cross-validation folds for this pipeline configuration
- Fit PCA separately to compute the actual number of components retained at that threshold

We generate two plots (as subplots in one figure):

- **RMSE vs. Variance Retained**: Illustrates how the average cross-validation RMSE changes as we increase the amount of variance retained by PCA
- **Components vs. Variance Retained**: Visualizes the dimensionality reduction effect of PCA

#### 4-fold Leave-One-Session-Out cross-validation of baseline models + PCA on *guided* dataset

In [ ]:
from sklearn.decomposition import PCA

# Evaluaton guided dataset
n_sessions, n_windows, n_channels, window_size = X_guided_windows_train_val_folds.shape

# Sweep PCA variance thresholds
variance_thresholds = np.arange(0.85, 0.991, 0.01)

for model_name, model in models_feat_sel.items():
    print('---------------------------------------------------------------------------')
    print(f'Evaluating PCA feature selection of the model {model_name} on the guided dataset...') 
    
    rmse_scores = []
    n_components_used = []

    for var in variance_thresholds:
        pipeline = Pipeline([
            ('td_feat', TimeDomainTransformer()),
            ('scale', StandardScaler()),
            ('pca', PCA(n_components=float(var), random_state=42)),
            ('reg', model)
        ])

        metric_fns = {'RMSE': RMSE}
        results = cross_validate_pipeline(
            pipeline,
            X_guided_windows_train_val_folds,
            Y_guided_labels_train_val_folds,
            metric_fns=metric_fns,
            verbose=0
        )

        # Fit once to extract # of components
        pipeline.fit(X_guided_windows_train_val_folds.reshape(-1, n_channels, window_size), 
                     Y_guided_labels_train_val_folds.reshape(-1, 51))
        n_comp = pipeline.named_steps['pca'].n_components_

        rmse_scores.append(results['avg_val_RMSE'])
        n_components_used.append(n_comp)

        # print(f"Variance: {var:.2f}, RMSE: {results['avg_val_RMSE']:.4f}, Components: {n_comp}")

    # Plots
    fig, axs = plt.subplots(1, 2, figsize=(14, 5))

    # Plot 1: RMSE vs Variance Retained
    axs[0].plot(variance_thresholds, rmse_scores, marker='o')
    axs[0].set_xlabel("PCA Variance Retained")
    axs[0].set_ylabel("Cross-Validation RMSE")
    axs[0].set_title(f"{model_name} - RMSE vs. Variance Retained - guided dataset")
    axs[0].grid(True)

    # Plot 2: #Components vs Variance Retained
    axs[1].plot(variance_thresholds, n_components_used, marker='s', color='green')
    axs[1].set_xlabel("PCA Variance Retained")
    axs[1].set_ylabel("# PCA Components Retained")
    axs[1].set_title(f"{model_name} - Components vs. Variance Retained - guided dataset")
    axs[1].grid(True)

    plt.tight_layout()
    plt.show()

In [ ]:
# Evaluaton freemoves dataset
n_sessions, n_windows, n_channels, window_size = X_freemoves_windows_train_val_folds.shape

# Sweep PCA variance thresholds
variance_thresholds = np.arange(0.85, 0.991, 0.01)

for model_name, model in models_feat_sel.items():
    print('---------------------------------------------------------------------------')
    print(f'Evaluating PCA feature selection of the model {model_name} on the freemoves dataset...') 
    
    rmse_scores = []
    n_components_used = []

    for var in variance_thresholds:
        pipeline = Pipeline([
            ('td_feat', TimeDomainTransformer()),
            ('scale', StandardScaler()),
            ('pca', PCA(n_components=float(var), random_state=42)),
            ('reg', model)
        ])

        metric_fns = {'RMSE': RMSE}
        results = cross_validate_pipeline(
            pipeline,
            X_freemoves_windows_train_val_folds,
            Y_freemoves_labels_train_val_folds,
            metric_fns=metric_fns,
            verbose=0
        )

        # Fit once to extract # of components
        pipeline.fit(X_freemoves_windows_train_val_folds.reshape(-1, n_channels, window_size), 
                     Y_freemoves_labels_train_val_folds.reshape(-1, 51))
        n_comp = pipeline.named_steps['pca'].n_components_

        rmse_scores.append(results['avg_val_RMSE'])
        n_components_used.append(n_comp)

        # print(f"Variance: {var:.2f}, RMSE: {results['avg_val_RMSE']:.4f}, Components: {n_comp}")

    # Plots
    fig, axs = plt.subplots(1, 2, figsize=(14, 5))

    # Plot 1: RMSE vs Variance Retained
    axs[0].plot(variance_thresholds, rmse_scores, marker='o')
    axs[0].set_xlabel("PCA Variance Retained")
    axs[0].set_ylabel("Cross-Validation RMSE")
    axs[0].set_title(f"{model_name} - RMSE vs. Variance Retained - freemoves dataset")
    axs[0].grid(True)

    # Plot 2: #Components vs Variance Retained
    axs[1].plot(variance_thresholds, n_components_used, marker='s', color='green')
    axs[1].set_xlabel("PCA Variance Retained")
    axs[1].set_ylabel("# PCA Components Retained")
    axs[1].set_title(f"{model_name} - Components vs. Variance Retained - freemoves dataset")
    axs[1].grid(True)

    plt.tight_layout()
    plt.show()

We observed that RMSE improves only marginally with initial increases in explained variance. However, beyond 0.90, further increases yield almost no RMSE improvements despite adding more components. Choosing much higher variance thresholds adds complexity without substantial accuracy gains, increasing overfitting risk and computation. 

Therefore, we selected an explained variance threshold near this 0.9 This balances:

* **Performance:** Accuracy close to the maximum.
* **Dimensionality Reduction:** Fewer features for a simpler model.
* **Generalization:** Reduced risk of overfitting.
* **Efficiency:** Lower computational cost.

In [ ]:
var = 0.9

baseline_pca_kr = Pipeline(
    [
        ('feature_extraction', TimeDomainTransformer()),
        ('scaler', StandardScaler()),
        ('feature_select', PCA(n_components=float(var), random_state=42)),
        ('regressor', KernelRidge(
            alpha = 0.01,
            gamma = 0.01,
            kernel='laplacian'))
    ]
)

baseline_pca_knn = Pipeline(
    [
        ('feature_extraction', TimeDomainTransformer()),
        ('scaler', StandardScaler()),
        ('feature_select', PCA(n_components=float(var), random_state=42)),
        ('regressor', KNeighborsRegressor(
            n_neighbors = 5))
    ]
)

baseline_pca_rf = Pipeline(
    [
        ('feature_extraction', TimeDomainTransformer()),
        ('scaler', StandardScaler()),
        ('feature_select', PCA(n_components=float(var), random_state=42)),
        ('regressor', RandomForestRegressor(
            n_estimators = 50,
            max_depth = 10))
    ]
)

#### 4-fold Leave-One-Session-Out cross-validation of baseline models + mMRM on *guided* dataset

In [ ]:
print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "baseline_pca_kr" on the guided dataset:')
results_guided_baseline_pca_kr = cross_validate_pipeline(
    pipeline = baseline_pca_kr,
    X_folds = X_guided_windows_train_val_folds,
    Y_folds = Y_guided_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "baseline_pca_knn" on the guided dataset:')
results_guided_baseline_pca_knn = cross_validate_pipeline(
    pipeline = baseline_pca_knn,
    X_folds = X_guided_windows_train_val_folds,
    Y_folds = Y_guided_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "baseline_pca_rf" on the guided dataset:')
results_guided_baseline_pca_rf = cross_validate_pipeline(
    pipeline = baseline_pca_rf,
    X_folds = X_guided_windows_train_val_folds,
    Y_folds = Y_guided_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

#### 4-fold Leave-One-Session-Out cross-validation of baseline models + PCA on *freemoves* dataset

In [ ]:
print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "baseline_pca_kr" on the freemoves dataset:')
results_freemoves_baseline_pca_kr = cross_validate_pipeline(
    pipeline = baseline_pca_kr,
    X_folds = X_freemoves_windows_train_val_folds,
    Y_folds = Y_freemoves_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "baseline_pca_knn" on the freemoves dataset:')
results_freemoves_baseline_pca_knn = cross_validate_pipeline(
    pipeline = baseline_pca_knn,
    X_folds = X_freemoves_windows_train_val_folds,
    Y_folds = Y_freemoves_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "baseline_pca_rf" on the freemoves dataset:')
results_freemoves_baseline_pca_rf = cross_validate_pipeline(
    pipeline = baseline_pca_rf,
    X_folds = X_freemoves_windows_train_val_folds,
    Y_folds = Y_freemoves_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

# Riemannian Geometry-based pipelines 

We will now enrich our arsenal of estimators, by developing three new pipelines based on the **Riemannian geometry of covariance matrices**.

The baseline pipelines developed above formulate predictions based on simple *time-domain features* extracted from EMG time windows. By contrast, we will now define estimators that formulate predictions based on *covariance matrices* extracted from EMG time windows.

As described above, a EMG time window comes in the form of a numpy array `X` of shape `(8, 500)`. By employing the transformer `pyriemann.estimation.Covariances()` from the freely available library `pyriemann`, we can extract a covariance matrix `X_covariance` of shape `(8, 8)`.

More precisely, we can think of `X` as a sample $\mathbf{x} = (x_1,...,x_8)$ from a random vector $\mathbf{X} = (X_1,...,X_8)$. We can then estimate its covariance $\text{Cov}(\mathbf{X}) = \text{Cov}(X_i, X_j)$ in terms of the *sample covariance matrix* of $\mathbf{x}$. The result is a $8\times8$ *symmetric and positive-semidefinite* matrix.

8x8 symmetric matrices form a vector space of dimension 36, parametrized by the entries of the upper-triangular part of the matrix. However, the Euclidean geometry of $\mathbb R^{36}$ is not well-suited to describe distances between covariance matrices. Indeed, the subset of *positive-semidefinite* matrices can be equipped with more appropriate natural geometries, encoded in the form of *Riemannian metrics*. The set of positive-semidefinite 8x8 symmetric matrices, equipped with a Riemannian metric, becomes a 36-dimensional *curved* manifold.

The library `pyriemann` exploits these metrics to implement classification and regression models. However, using the intrinsic Riemannian geometry of this manifold can be computationally quite expensive. For this reason, it is convenient to take a first-order approximation of the manifold, by *projecting orthogonally* the covariance matrices to a tangent space. The tangent space at a point is a 36-dimensional *Euclidean* vector space, representing a *linear* approximation of the curved manifold near the given point. Projections to the tangent space thus provide features in the form of vectors in Euclidean space, on which standard regressors and estimators can be used.

This projection procedure is implemented in `pyriemann` in the form of a sklearn-compatible transformer. Specifically, an instance of the class `pyriemann.tangentspace.TangentSpace` has:

- a `.fit(X, y)` method, where `X` is a numpy array of covariance matrices (shape `(n_matrices, n_channels, n_channels)`), with `y` not used but included for compatibility with sklearn; this method computes and stores internally the Riemannian average of the covariance matrices;

- a `.transform(X)` method, which (if called after `.fit()`) takes in input a numpy array of covariance matrices (shape `(n_matrices, n_channels, n_channels)`) and returns the orthogonal projections of these matrices to the tangent space at the Riemannian average (shape `(n_matrices, n_channels * n_channels)`).

Our pipelines will follow a common pipeline structure:

1. **Covariance Estimation**  
   `pyriemann.estimation.Covariances()`  
   - Computes the covariance matrix from EMG channels per time window.

2. **Tangent Space Projection**  
   `pyriemann.tangentspace.TangentSpace(metric='riemann', tsupdate=True)`  
   - Projects the SPD matrices onto the tangent space at the Riemannian mean.
   - `tsupdate=True` allows updating the tangent space mean at transform time.

3. **Feature Scaling**  
   `StandardScaler()`
   - Standard normalization to mean 0 and unit variance.

4. **Regression Model**  
   - Varies across pipelines below.

As regression model we use again:
   
   - **Kernel Ridge Regression**

   - **K-Nearest Neighbors Regression**

   - **Random Forest Regression**

The reasons for this particular choice are the same explained in the section describing the baseline pipelines: after some experiments, these models perform reasonably well compare to the others, and they are sufficiently different to ensure good improvement when considered together in an ensemble model.

In [ ]:
import pyriemann
import pyriemann.regression

In [ ]:
# Riemannian-based pipelines
riem_kr = Pipeline(
    [
        ('feature_extraction', pyriemann.estimation.Covariances()),
        ('transformation', pyriemann.tangentspace.TangentSpace(
            metric = 'riemann',
            tsupdate = True)),
        ('scaler', StandardScaler()),
        ('regressor', KernelRidge(
            alpha = 0.01,
            gamma = 0.01,
            kernel='laplacian'))
    ]
)

riem_knn = Pipeline(
    [
        ('feature_extraction', pyriemann.estimation.Covariances()),
        ('transformation', pyriemann.tangentspace.TangentSpace(
            metric = 'riemann',
            tsupdate = True)),
        ('scaler', StandardScaler()),
        ('regressor', KNeighborsRegressor(
            n_neighbors = 5))
    ]
)

riem_rf = Pipeline(
    [
        ('feature_extraction', pyriemann.estimation.Covariances()),
        ('transformation', pyriemann.tangentspace.TangentSpace(
            metric = 'riemann',
            tsupdate = True)),
        ('scaler', StandardScaler()),
        ('regressor', RandomForestRegressor(
            n_estimators = 50,
            max_depth = 10))
    ]
)

As for the baseline models, we cross-validate these models on the train-val part of the dataset, by means of **Leave-One-Session-Out 4-fold cross-validation**.

#### 4-fold Leave-One-Session-Out cross-validation of Riemannian models on *guided* dataset

In [ ]:
print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "riem_kr" on the guided dataset:')
results_guided_riem_kr = cross_validate_pipeline(
    pipeline = baseline_kr,
    X_folds = X_guided_windows_train_val_folds,
    Y_folds = Y_guided_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "riem_knn" on the guided dataset:')
results_guided_riem_knn = cross_validate_pipeline(
    pipeline = baseline_knn,
    X_folds = X_guided_windows_train_val_folds,
    Y_folds = Y_guided_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "riem_rf" on the guided dataset:')
results_guided_riem_rf = cross_validate_pipeline(
    pipeline = baseline_rf,
    X_folds = X_guided_windows_train_val_folds,
    Y_folds = Y_guided_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

#### 4-fold Leave-One-Session-Out cross-validation of Riemannian models on *freemoves* dataset

In [ ]:
print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "riem_kr" on the freemoves dataset:')
results_freemoves_riem_kr = cross_validate_pipeline(
    pipeline = baseline_kr,
    X_folds = X_freemoves_windows_train_val_folds,
    Y_folds = Y_freemoves_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "riem_knn" on the freemoves dataset:')
results_freemoves_riem_knn = cross_validate_pipeline(
    pipeline = baseline_knn,
    X_folds = X_freemoves_windows_train_val_folds,
    Y_folds = Y_freemoves_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "riem_rf" on the freemoves dataset:')
results_freemoves_riem_rf = cross_validate_pipeline(
    pipeline = baseline_rf,
    X_folds = X_freemoves_windows_train_val_folds,
    Y_folds = Y_freemoves_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

# Domain-Adversarial Neural Network (DANN)

In EMG-based regression, **inter-session variability** presents a key challenge to generalisation. Signals can differ substantially across recording sessions due to factors such as:

- Electrode placement drift
- Muscle fatigue or tension variation
- Electrical noise or impedance shifts
- Skin conductivity changes over time

Even when gestures are performed identically, the signal shapes and amplitudes can vary — this is clearly visible when comparing sessions directly.

## Empirical check of session variability

To substantiate the use of domain-adversarial learning, we first note that our **baseline models exhibit poor cross-session generalisation** showing **one fold significantly underperforming the others**. This suggests that the model overfits to session-specific features.
Rather than relying solely on loss metrics, we tried visually highlight this phenomenon. We propose computing a **per-session signal profile summary**, using variance or magnitude-based time features (e.g., mean absolute value, waveform length, etc.) aggregated over all windows and channels. This provides a condensed summary of the **typical signal signature** for each session.

**Suggested visualisation:** - TEMPORARY
- A grid of plots — one per session — each showing a compact summary (e.g., channel-wise MAV or RMS curves over time)
- This allows direct visual comparison of **how sessions differ globally**, while avoiding the complexity of deeper classifiers

Such a summary figure helps confirm that:
- Each session indeed has a **distinct signal signature**
- Domain shifts are large enough to justify using a **domain-adversarial approach**

In this context, DANN becomes essential: it penalises representations that retain session identity and enforces learning of **shared signal structure**, not session-specific noise. Rather than rely on classical regularisation alone, we use **domain-adversarial training** to penalise representations that encode session-specific information.

The core idea is: 
- If a feature extractor allows a domain classifier to easily identify which session a signal comes from, then it is learning features that are not session-invariant.  
- DANN flips the gradients from the domain classifier to *discourage* such behaviour.

By integrating a **domain classifier and gradient reversal layer**, we force the network to **learn features that generalise across sessions**, leading to more robust predictions even on unseen recordings.

According to the literature CITATION OF PAPER convolutional Neural Networks (CNNs) are well-suited to modelling **temporal and spatial patterns** in time-series data like EMG. The reasons are:

- They can detect **local patterns** (e.g. bursts or waves) that correspond to muscle activation
- They **share weights** across time, ensuring translation-invariance
- They efficiently extract features over long time windows (e.g. 500ms) without exploding parameter counts

This makes CNNs particularly effective for EMG feature extraction, especially when combined with DANN to learn session-agnostic representations.

## DANN Architecture

The DANN model used in this project consists of three key components:

>  ### 1. Feature Extractor (CNN)
> 
> - **Purpose:** Converts the raw multi-channel EMG signal into a compact latent representation that captures relevant temporal features.
> - **Input:** A tensor of shape `(batch_size, channels, time)`, typically including the raw signal and its first derivative.
> - **Architecture:** A stack of 1D convolutional layers with ReLU activation, batch normalization, dropout, and pooling.
> - **Output:** A feature vector of fixed size (e.g. 128-dim), serving as the input to both the regressor and the domain classifier.
> 
> This block is responsible for **capturing spatial and temporal correlations** in EMG signals while being general enough to apply across recording sessions.
> 
> 
> ### 2. Regressor Head
> 
> - **Purpose:** Predicts the 51-dimensional vector of joint angles from the extracted feature representation.
> - **Input:** The latent feature vector from the feature extractor.
> - **Output:** A continuous prediction for each joint angle.
> 
> While this module is evaluated using **Mean Squared Error (MSE)** on joint angle predictions, it is not trained in isolation — it is optimised jointly as part of a broader loss function that includes domain adaptation objectives.
> 
> 
> ### 3. Domain Classifier + Gradient Reversal Layer (GRL)
> 
> - **Purpose:** Attempts to classify which session the input signal came from.
> - **Architecture:** A lightweight classifier (e.g., MLP) producing a probability distribution over the session domain labels.
> - **Gradient Reversal Layer (GRL):** During backpropagation, the GRL inverts the gradient signal coming from the domain classifier. This encourages the feature extractor to **remove session-specific information**.
> 
> By penalising features that allow easy domain classification, the feature extractor is encouraged to learn **session-invariant representations**, improving generalisation.

### Training Strategy

The DANN model is trained using a **composite loss function** that balances prediction accuracy and domain invariance. Specifically, the model minimises the following total loss:

$$
\mathcal{L}_{\text{total}} = \mathcal{L}_{\text{reg}} + \lambda \cdot \mathcal{L}_{\text{domain}} - \gamma \cdot \mathcal{H}(\text{domain\_pred})
$$

Where:
- $\mathcal{L}_{\text{reg}}$: Mean Squared Error (MSE) between predicted and true joint angles  
- $\mathcal{L}_{\text{domain}}$: Cross-entropy loss on predicted session (domain)  
- $\mathcal{H}$: Shannon entropy of the domain prediction distribution (optional regulariser)  
- $\lambda$: Adversarial weighting factor (annealed from 0 to 1)  
- $\gamma$: Entropy regularisation coefficient (typically small, e.g. 0.05–0.1)

At each iteration during training:

1. The EMG input $x$ is passed through the **feature extractor**, producing latent features.
2. These features are passed to:
   - The **regressor**, which outputs joint angle predictions $\hat{y}$
   - The **domain classifier**, via a **Gradient Reversal Layer**, which outputs session prediction logits
3. The three loss components are computed and combined into the total loss.
4. A single backward pass updates all parameters jointly, with the GRL flipping gradients flowing into the feature extractor from the domain classifier.

The python implementation od the DANN model is done creating compatible torch modules: 

- `class DANNModel(nn.Module)`: Defines the core Domain-Adversarial Neural Network architecture. It encapsulates the three key submodules:

   1. `ConvFeatureExtractor`:  
      CNN-based block transforming EMG windows of shape `(batch_size, 8, 500)` into compact 128-dim features.
   2. `RegressorHead`:  
      Fully connected layers that map latent features to the 51 joint-angle outputs.
   3. `DomainDiscriminator`:  
      Lightweight classifier (MLP) to predict session identity. Connected via a **Gradient Reversal Layer (GRL)** to encourage domain-invariant feature learning.

<!-- **Key Parameters:**

- `lambda_grl` *(float)*: Weight of gradient reversal in the adversarial loss. Passed at each forward pass.
- `num_domains` *(int)*: Number of different session labels for the domain classifier.
- `output_dim` *(int)*: Number of regression outputs (default is 51 joint angles). -->

- `class DANNTrainer`: Manages the full training loop for DANN, including:

   - Gradient reversal and joint optimisation of regression + domain losses
   - Cross-session early stopping
   - Composite loss:  
   $$
   \mathcal{L}_{\text{total}} = \mathcal{L}_{\text{reg}} + \lambda \cdot \mathcal{L}_{\text{domain}} - \gamma \cdot \mathcal{H}
   $$

<!-- **Key Arguments:**

- `model` *(nn.Module)*: Must follow the DANN interface (e.g., return both regression and domain outputs).
- `train_loader`, `val_loader`: PyTorch `DataLoader` objects for training/validation.
- `lambda_grl`: Strength of the gradient reversal loss component (annealed across epochs).
- `gamma_entropy`: Optional entropy regularisation to encourage uniform domain predictions.
- `max_epochs`, `patience`: Control training loop and early stopping.
- `device`: Typically `"cuda"` or `"cpu"`.

**Core Methods:**

- `.train()`: Trains with early stopping on validation RMSE.
- `.predict(data_loader)`: Forward pass on new data.
- `.evaluate()`: Computes RMSE on validation set.
- `.evaluate_domain_on_train()`: Returns accuracy of domain predictions on training set — useful to diagnose invariance. -->

To be consistent with our cross-validation strategy, we also created a `cross_validate_dann` function to automates the session-wise **leave-one-session-out** cross-validation for DANN training. The function works in following way:

1. Splits the 5 sessions into 4 training and 1 validation.
2. Constructs appropriate session IDs for domain classification.
3. Trains a new `DANNModel` per fold with fresh `DANNTrainer`.
4. Aggregates RMSE scores across folds.

In [ ]:
from config.dann import DataLoader, DANNModel, DANNTrainer, DANNWindowTensor, DANNRegressor, DANNConfig, DANNRunner, torch, TemporalDANNModel

#### 4-fold Leave-One-Session-Out cross-validation of DANN model on *guided* dataset

In [ ]:
config_guided = DANNConfig(
    batch_size=256,
    max_epochs=40,
    patience=10,
    learning_rate=1e-3,
    lambda_grl=1.0,
    gamma_entropy=0.05,
    num_domains=4,
    output_dim=51,
    device="cuda" if torch.cuda.is_available() else "cpu",
    verbose=1  # verbose=2 for epoch logs
)


runner_guided = DANNRunner(
    model_class=lambda **kwargs: DANNModel(**kwargs),
    dataset_class=DANNWindowTensor,
    config=config_guided
)

results_guided_dann = runner_guided.cross_validate(
    X_guided_windows_train_val_folds, 
    Y_guided_labels_train_val_folds
)

#### 4-fold Leave-One-Session-Out cross-validation of DANN model on *freemoves* dataset

In [ ]:
config_freemoves = DANNConfig(
    batch_size=256,
    max_epochs=40,
    patience=10,
    learning_rate=1e-3,
    lambda_grl=1.0,
    gamma_entropy=0.05,
    num_domains=4,
    output_dim=51,
    device="cuda" if torch.cuda.is_available() else "cpu",
    verbose=1  # verbose=2 for epoch logs
)


runner_freemoves = DANNRunner(
    model_class=lambda **kwargs: DANNModel(**kwargs),
    dataset_class=DANNWindowTensor,
    config=config_freemoves
)

results_freemoves_dann = runner_freemoves.cross_validate(
    X_freemoves_windows_train_val_folds, 
    Y_freemoves_labels_train_val_folds
)

# TempDANN

TO DO .......

#  Ensemble

Design and implement two ensembling strategies that combine the predictions of all individual regression models you implemented, each independently trained on distinct feature representations of the EMG signals. The two required ensembling approaches are the following:
- Compute the average of the predicted values
- Use a meta-learner strategy (often referred to as stacking), in which a separate model—trained on the outputs of the base learners—generates the ensemble prediction
For each ensemble strategy, compare and document the ensemble’s regression performance against each of its constituent base models using consistent regression metrics. Additionally, for the metalearner strategy, determine and discuss the relative contribution of each base model to the final ensemble prediction. Discuss how the bias-variance tradeoff relates to the observed (or expected) evolution of performance.

## Ensemble Method: Voting Regressor

To enhance the robustness and accuracy of our predictions, we employ an ensemble learning technique using a Voting Regressor. The idea behind ensembling is simple: different models may capture different aspects of the signal, and by aggregating their predictions, we can reduce variance, mitigate individual model weaknesses, and improve overall robustness and accuracy.

We implement a **Voting Regressor**, which performs prediction by taking a **weighted average** of the outputs of multiple base regressors. If we denote the predictions of $M$ models as $\hat{y}_1, \hat{y}_2, \ldots, \hat{y}_M$, then the final ensemble prediction is computed as:

$$
\hat{y}_{\text{ensemble}} = \frac{1}{M} \sum_{m=1}^{M} \hat{y}_m
$$]

Our implementation of the Voting Regressor is as a custom scikit-learn-compatible class `VotingRegressor`, which fits each constituent baseline pipeline individually and then combines their predictions at inference time. The ensemble combines both time-domain and Riemannian geometry-based pipelines. Each included model brings a different inductive bias or mathematical structure to the task:

* `baseline_kr`: Kernel Ridge Regression applied to time-domain features.
* `baseline_rf`: Random Forest Regression applied to time-domain features.
* `riem3`, `riem4`: Riemannian geometry-based pipelines utilizing tangent space projections of covariance matrices.

These pipelines were chosen based on their individual performance and the diversity of their approaches to ensure a well-rounded ensemble.

The final ensemble is instantiated as:
```python
voting_estimator = VotingRegressor(
    estimators = [
        baseline_guided_kr,
        baseline_guided_rf,
        riem3,
        riem4
    ]
)
```

In [ ]:
from config.regressors import VotingRegressor

# TO CHECK: ENSEMBLE VOTING CODE

voting_estimator = VotingRegressor(
    estimators = [
        baseline_kr,
        baseline_knn,
        baseline_rf,
        riem_kr,
        riem_knn,
        riem_rf
    ]
)

#### 4-fold Leave-One-Session-Out cross-validation of Voting Regressor model on *guided* dataset

In [ ]:
print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "voting_estimator" on the guided dataset:')
results_guided_voting = cross_validate_pipeline(
    pipeline = voting_estimator,
    X_folds = X_guided_windows_train_val_folds,
    Y_folds = Y_guided_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

#### 4-fold Leave-One-Session-Out cross-validation of Voting Regressor model on *freemoves* dataset

In [ ]:
print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "voting_estimator" on the freemoves dataset:')
results_freemoves_voting = cross_validate_pipeline(
    pipeline = voting_estimator,
    X_folds = X_freemoves_windows_train_val_folds,
    Y_folds = Y_freemoves_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

## Ensemble Method: Stackig Regressor

TO DO: ENSEMBLE STACKING DESCRIPTION

In [ ]:
from config.regressors import StackingRegressor

stacking_estimator = StackingRegressor(
    estimators = [
        baseline_kr,
        baseline_knn,
        baseline_rf,
        riem_kr,
        riem_knn,
        riem_rf
    ],
    end_estimator = RandomForestRegressor(
        n_estimators = 50,
        max_depth = 10)
)

#### 4-fold Leave-One-Session-Out cross-validation of Stacking Regressor model on *guided* dataset

In [ ]:
print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "stacking_estimator" on the guided dataset:')
results_guided_voting = cross_validate_pipeline(
    pipeline = stacking_estimator,
    X_folds = X_guided_windows_train_val_folds,
    Y_folds = Y_guided_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

#### 4-fold Leave-One-Session-Out cross-validation of Stacking Regressor model on *freemoves* dataset

In [ ]:
print('---------------------------------------------------------------------------')
print('Cross-Validation of the pipeline "stacking_estimator" on the freemoves dataset:')
results_freemoves_voting = cross_validate_pipeline(
    pipeline = stacking_estimator,
    X_folds = X_freemoves_windows_train_val_folds,
    Y_folds = Y_freemoves_labels_train_val_folds,
    metric_fns = {'RMSE': RMSE, 'NMSE': NMSE},
    n_folds = 4,
    # verbose 0: silent mode; verbose 1: only average across folds values printed; verbose 2: results of all folds printed
    verbose = 1
)

#  Extra

## Filter

## Cross validation full data

## Fancy 3D plots

## Autoencoder

# Final models